In [3]:
############# working ###################



import cv2
import numpy as np
import imutils

def detect_shape(c):
    shape = "unidentified"
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.03 * peri, True)

    if len(approx) == 2:
        shape = "line"
    elif len(approx) >= 3 and cv2.isContourConvex(approx):
        shape = "dotted line"
    elif len(approx) == 4:
        (x, y, w, h) = cv2.boundingRect(approx)
        ar = w / float(h)
        shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"

    return shape

image = cv2.imread("sequence_diagram.png")

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 90, 150, 3)
contours = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)

line_count = 0  # Counter for lines
dotted_line_count = 0  # Counter for dotted lines
rectangle_count = 0  # Counter for rectangles

# Create a dictionary to store the in and out message counts for each object
object_messages = {}

for contour in contours:
    shape = detect_shape(contour)
    M = cv2.moments(contour)

    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        contour = contour.astype("float")
        contour = contour.astype("int")

        if shape == "line":
            cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
            cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
                         0.5, (0, 0, 0), 2)
            line_count += 1  # Increment the line counter
            
            # Determine the sender and receiver objects based on the x-coordinate of the line
            if cX < image.shape[1] // 2:
                sender_object = 4
                receiver_object = 5
            else:
                sender_object = 3
                receiver_object = 5
                
            # Increment the out message count for the sender object
            object_messages.setdefault(sender_object, {"in_messages": 0, "out_messages": 0})
            object_messages[sender_object]["out_messages"] += 1
            
            # Increment the in message count for the receiver object
            object_messages.setdefault(receiver_object, {"in_messages": 0, "out_messages": 0})
            object_messages[receiver_object]["in_messages"] += 1
        
        elif shape == "dotted line":
            cv2.drawContours(image, [contour], -1, (0, 0, 255), 2)
            cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
                         0.5, (0, 0, 0), 2)
            dotted_line_count += 1  # Increment the dotted line counter
            
            # Determine the sender and receiver objects based on the x-coordinate of the dotted line
            if cX < image.shape[1] // 2:
                sender_object = 4
                receiver_object = 5
            else:
                sender_object = 3
                receiver_object = 5
            
            # Increment the in message count for the sender object
            object_messages.setdefault(sender_object, {"in_messages": 0, "out_messages": 0})
            object_messages[sender_object]["in_messages"] += 1
            
            # Increment the out message count for the receiver object
            object_messages.setdefault(receiver_object, {"in_messages": 0, "out_messages": 0})
            object_messages[receiver_object]["out_messages"] += 1
                
        elif shape == "square" or shape == "rectangle":
            cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
            cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
                         0.5, (0, 0, 0), 2)
            rectangle_count += 1  # Increment the rectangle counter

# Print the number of detected shapes
print("Number of messages:", line_count)
print("Number of return messages:", dotted_line_count)
print("Number of objects:", rectangle_count // 2)

# Print the in and out message counts for each object
for object_number, message_count in object_messages.items():
    print("Object", object_number)
    print("In Messages:", message_count["in_messages"])
    print("Out Messages:", message_count["out_messages"])

# Show the output image with the shapes
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Number of messages: 7
Number of return messages: 1
Number of objects: 1
Object 3
In Messages: 1
Out Messages: 4
Object 5
In Messages: 7
Out Messages: 1
Object 4
In Messages: 0
Out Messages: 3
